# Language Knowledge (Vocabulary)
Duration: 30 minutes
Content: This section tests your knowledge of Japanese vocabulary, including kanji readings, orthography, word formation, contextually-defined expressions, paraphrases, and usage
It mainly composes following five categories:
- ``Reading Kana`` (Pronunciation Questions): Given a kanji word, choose the correct kana reading.
- `Writing Kanji` (Writing Questions): Given a word written in kana, choose the correct kanji representation.
- `Word Meaning` Selection (Vocabulary Understanding): Choose the most suitable word to fill in the sentence from four options.
- `Synonym Replacement`: Select a word that has the same or similar meaning as the underlined word.
- `Vocabulary Usage`: Assess the usage of words in actual contexts, choosing the most appropriate word usage, including some common Japanese expressions or fixed phrases.

In [23]:
import pandas as pd
import json
import os
import pickle
import re
import uuid
from typing import *
from langchain_openai import AzureOpenAI,AzureChatOpenAI
from langchain_anthropic import ChatAnthropic
from langchain_core.prompts import ChatPromptTemplate
from dotenv import load_dotenv
from langchain_aws import ChatBedrock
from langchain.embeddings.base import Embeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.embeddings import HuggingFaceEmbeddings
# from langchain_community.embeddings import XinferenceEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
from typing import Annotated, Literal, Sequence
from typing_extensions import TypedDict
from IPython.display import display, Markdown, Latex
from langchain import hub
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from typing import Annotated, Sequence
from typing_extensions import TypedDict
from langchain_core.messages import BaseMessage,RemoveMessage,HumanMessage,AIMessage,ToolMessage
from langgraph.graph.message import add_messages
from pydantic import BaseModel, Field
from langgraph.graph import END, StateGraph, START
from langgraph.prebuilt import ToolNode
from langgraph.prebuilt import tools_condition
from langgraph.checkpoint.memory import MemorySaver
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field, validator
from typing import List, Optional

load_dotenv()

True

In [24]:
# Import N3 Vocabulary
file_path = '../Vocab/n3.csv'
# Read the CSV file
data = pd.read_csv(file_path)
words = data.iloc[:, :2].sample(frac=1).reset_index(drop=True)
# Display the content of the CSV file
words.head()

,expression,reading
0,名人,めいじん
1,重体,じゅうたい
2,楽,らく
3,加わる,くわわる
4,小包,こづつみ


#### load Models

In [25]:
# azure_llm = AzureChatOpenAI(
#     azure_endpoint="https://tooldev-openai.openai.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2025-01-01-preview",
#     api_key=os.environ["AZURE_API_KEY"],
#     model_name="gpt-4o",
#     api_version="2025-01-01-preview",
#     temperature=0.5,
# )

In [26]:
aws_llm = ChatBedrock(
    model_id="us.anthropic.claude-3-7-sonnet-20250219-v1:0",
     # model_id="us.anthropic.claude-3-5-haiku-20241022-v1:0",
    model_kwargs=dict(temperature=0.3),
    region = "us-east-2",
    aws_access_key_id=os.environ["AWS_ACCESS_KEY_ID"],
    aws_secret_access_key=os.environ["AWS_SECRET_ACCESS_KEY"],
)

## Question Explore

# Kanji 读假名（读音问题）

In [27]:
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END
from IPython.display import Image, display


# Graph state
class QuestionState(TypedDict):
    topic: str
    question: str
    improved_question: str
    final_question: str


kanji_example = """
問題 1

（　　）に入れるのに最もよいものを、1・2・3・4から一つ選びなさい。

1.

私は、自分の作ったパンを多くたくさんの人（　）食べてほしいと思って、パン屋を始めた。
	1.	は
	2.	に
	3.	まで
	4.	なら

2.（学校にて）

学生：「先生、今、よろしいですか。英語の発表（　）、ちょっと相談したいのですが。」
先生：「ええ、いいですよ。」
	1.	にとって
	2.	によると
	3.	のことで
	4.	のは

3.

いつもは勉強を2時間以上かかるが、今日は1時間（　）終わりそうだ。
	1.	くらい
	2.	ころで
	3.	ぐらい
	4.	ぐらいで

4.

母：「えっ、（　）ご飯食べたばかりなのに、もうおなかすいたの？」
	1.	そろそろ
	2.	だんだん
	3.	さっき
	4.	ずっと

5.

大事なレシートをズボンのポケットに（　）洗濯してしまった。
	1.	入れたまま
	2.	入ったまま
	3.	入れている間
	4.	入っている間

6.（駅のホームにて）

「急げ、9時の特急に間に合うかもしれないし、走ろうか。」
「いや、（　）もう間に合わないと思う。次の電車にしよう。」
	1.	走ってて
	2.	走ってるよ
	3.	走らさきゃ
	4.	走っちゃって

7.

私はよくインターネットで物を買い替えるが、掃除機は壊れたら、実際に（　）買いたいものだ。
	1.	見てないと
	2.	見ておきたくなった
	3.	見てから
	4.	見ておいて

8.（料亭にて）

（体を丸めてお辞儀をして）「おいしそうな料理ですね。」
店員：「どうぞたくさん（　）ください。」
	1.	召し上がって
	2.	おっしゃって
	3.	なおって
	4.	いらっし

9.

A：「最近、寒くなって（　）ね。」
B：「ええ、今日は特に冷えますね。」
	1.	いました
	2.	ありました
	3.	いきました
	4.	きました

10.（大学にて）

A：「日曜日の留学生交流会、どうだった？」
B：「楽しかったよ。初めてだったからちょっと緊張したけど、新しい友達もできたし。」
	1.	行ってよかったよ
	2.	行こうかと思うよ
	3.	行きたかったなあ
	4.	行けたらいいなあ

11.（大学の事務所で）

学生：「すみません、ペンを（　）。」
事務所の人：「あ、はい、これを使ってください。」
	1.	お貸しできますか
	2.	お貸しいたしますか
	3.	貸したらいかがですか
	4.	貸していただけませんか

12.（家にて）

娘：「ちょっと駅前の本屋に行ってくるね。」
父：「雨が降っているし、車で（　）？」
娘：「いいの？ありがとう。」
	1.	送っててない
	2.	送ってこようか
	3.	送ってあげない
	4.	送ってあげようか

13.（会社にて）

「中山さん、今、ちょっといいですか。」
中山：「あ、ごめんなさい、これからABC銀行に（　）、戻ってきてからでもいいですか。」
	1.	行くところだからです
	2.	行くとこなんです
	3.	行っているところだからです
	4.	行っているところなんです
"""

print(words)

# Nodes
def generate_question(state: QuestionState):
    """First LLM call to generate initial question"""

    msg = aws_llm.invoke(f"You are a japanese teacher. Your job is to write 13 grammar questions for candidate to identify kanji characters and their meanings for a JLPT N3 level exam paper by topics that user provided in the topic list. This includes recognizing kanji in context, such as in sentences or phrase. Don't replace kanji part and only substitute in the options. The JLPT exam paper includes a mix of easy, moderate, and difficult questions to accurately assess the test-taker's proficiency across different aspects of the language."
            f"The vocabulary should be restricted to N3 level, you can refer to the vocabulary in the word list, choosing random words for the questions"
            f"please refer the question examples following the formal exam paper"
            f"append the correct answer and explanation of main challenges and why teacher asks this question to candidate in chinese at each question"
            f"Finally, beautify markdown format"
            f"topic list: {state['topic']}"
            f"word list: {words}"
            f"formal exam paper: {kanji_example}")
    
    return {"question": msg.content}


# Build workflow
kanji_workflow = StateGraph(QuestionState)

# Add nodes
kanji_workflow.add_node("generate_question", generate_question)

# Add edges to connect nodes
kanji_workflow.add_edge(START, "generate_question")
kanji_workflow.add_edge("generate_question", END)

# Compile
kanji_graph = kanji_workflow.compile()


     expression reading
0            名人    めいじん
1            重体   じゅうたい
2             楽      らく
3           加わる    くわわる
4            小包    こづつみ
...         ...     ...
2134         回復    かいふく
2135          偶      たま
2136         疲れ     つかれ
2137          孫      まご
2138          球      たま

[2139 rows x 2 columns]


In [28]:
# Show workflow
#display(Image(kanji_graph.get_graph().draw_png()))

In [29]:
# Invoke
kanji = kanji_graph.invoke({"topic": "店で価格を尋ねる | レストランで食べ物を注文する | 交通手段について話す| 天気の状況について話す | 趣味について話す | 店で商品の特徴や価格を尋ねる | スーパーマーケットで買い物をする | レストランで食べ物や飲み物を注文する | カフェでテイクアウトを頼む | コンビニで支払い方法について質問する |  ネットショッピングで商品を選ぶ"})
display(Markdown(kanji["question"]))

# JLPT N3 漢字問題

## 問題 1

（　　）に入れるのに最もよいものを、1・2・3・4から一つ選びなさい。

1. この**店**では、さまざまな（　　）の商品を取り扱っています。
   1. 価格
   2. 代金
   3. 料金
   4. 金額

**正解：1. 価格**

*解説：この問題は「店で価格を尋ねる」というトピックに関連しています。「価格」は商品の値段を表す最も一般的な言葉です。「代金」は支払う金額、「料金」はサービスに対して支払う金額、「金額」は総額を意味します。店の商品について話す場合は「価格」が最も適切です。この問題で候補者の語彙の正確さを測ります。*

## 問題 2

2. **レストラン**で（　　）を見ながら、何を食べるか決めましょう。
   1. 注文
   2. 献立
   3. メニュー
   4. 料理

**正解：3. メニュー**

*解説：「レストランで食べ物を注文する」というトピックに関連する問題です。レストランで食事を選ぶ際に見るのは「メニュー」が最も適切です。「注文」は注文する行為、「献立」は家庭や施設での食事計画、「料理」は調理された食べ物を指します。この問題は日常的な場面での適切な語彙選択能力を測ります。*

## 問題 3

3. 明日の**天気**（　　）によると、午後から雨が降るそうです。
   1. 予報
   2. 情報
   3. 状態
   4. 発表

**正解：1. 予報**

*解説：「天気の状況について話す」というトピックに関連する問題です。「天気予報」は気象状況の予測を表す最も適切な表現です。「天気情報」も使いますが、「天気予報」ほど一般的ではありません。「天気状態」「天気発表」という表現はあまり使いません。この問題は天気に関する表現の理解度を測ります。*

## 問題 4

4. 彼は**交通**（　　）が便利な場所に住んでいます。
   1. 手段
   2. 機関
   3. 方法
   4. 道具

**正解：2. 機関**

*解説：「交通手段について話す」というトピックに関連する問題です。「交通機関」は電車やバスなどの公共交通を指す表現です。「交通手段」も使いますが、この文脈では「交通機関が便利」という表現が自然です。この問題は交通に関する語彙の適切な使用を測ります。*

## 問題 5

5. 私の**趣味**は写真（　　）です。休日にはカメラを持って出かけます。
   1. 撮影
   2. 撮る
   3. 取る
   4. 写す

**正解：1. 撮影**

*解説：「趣味について話す」というトピックに関連する問題です。写真を趣味とする場合、「写真撮影」という表現が最も適切です。「撮る」は動詞の基本形、「取る」は間違った漢字、「写